## Installing Sentence Transsformer and other models/frameworks

In [1]:
!pip install sentence_transformers

# Kindly add all your installations and versions if any in this cell.


[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Importing necessary libraries. 
In the final version all imports should be stricly enlisted here.

In [2]:
import pandas as pd
import numpy as np
import spacy
from scipy import stats
from sklearn import linear_model

from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

import torch 
from torch.utils.data import DataLoader

## Load dataset: 7 marks
1 Download and unzip the dataset from this link http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz  **1 mark**

2 Complete the code in `read_sts_csv()`. **4.5 marks**

3 Create 3 dataframes one each for train, test and val and print their final shapes. **1.5 marks**

In [8]:
INPUT_PATH = 'stsbenchmark/'

def read_sts_csv(dataset_type="train", columns=['source', 'type', 'year', 'id', 'score', 'sent_a', 'sent_b']):
  path = INPUT_PATH + "sts-"+ dataset_type + ".csv"
  """
  Take the input path and return the dataframe
  """
  # Open File as Text File
  with open(path, 'r', encoding='utf-8') as f:
    # Read the file as a list of lines
    lines = f.readlines()

  output = []
  for line in lines:
    # Split the line by tab
    line = line.strip().split('\t')
    # Append the line to output
    output.append(line[:len(columns)])

  # Convert the output to a dataframe
  df = pd.DataFrame(output, columns=columns)
  return df

# df_<dataset_type> = read_sts_csv(dataset_type) # create the train, dev and test dataframes
df_train = read_sts_csv("train")
df_dev = read_sts_csv("dev")
df_test = read_sts_csv("test")

## Hyperparameters: 5 Marks
Update this cell with you choosen parameters except, NUM_EPOCHS

In [ ]:
NON_CONEXTUAL_MODEL_TYPE = <NON_CONEXTUAL_MODEL_TYPE>
CONEXTUAL_MODEL_TYPE = <CONEXTUAL_MODEL_TYPE>
HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL = <HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL> # USE THE HUGGAING FACE VERSION OF SENTENCE_TRANSFORMER_TYPE
INPUT_PATH = <INPUT_FOLDER_PATH>
BATCH_SIZE = <BATCH SIZE>
OUT_DIM_DENSE = <OUT_DIM_DENSE>
NUM_EPOCHS = 2 ## THIS IS FIXED DO NOT CHANGE

# You are free to add your own hyperparameters as well.

## CONFIGURATION 1: Non-contextual Embeddings + ML Regression: 8 marks
1 Load the non-contextual embedding model in variable `non_cont_model1`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model1()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model1`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model1`. **1.5 mark**



In [ ]:
def get_feature_model1(data_frame):
  """
  Input a data frame and return the embedding vectors for the each sentence column using non_cont_model1,
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """

# non_cont_model1 = 

# feature_1_<dataset_type>, feature_2_<dataset_type> = get_feature_model1(data_frame)
# X_<dataset_type>, Y_<dataset_type> = 
# Initiate a regression model and train it.
# Print spearmanr correlation on the predicted output of the dev and test sets.

## CONFIGURATION 2: Contextual Embeddings + ML Regression: 7 marks
1 Load the contextual embedding model in variable `non_cont_model2`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model2()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model2`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model2`. **1.5 mark**

Useful references: https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [ ]:
def get_feature_model2(data_frame):
  """
  Input a data frame and return the embedding vectors for the each sentence column using model2,
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """

# non_cont_model2 = 

# feature_1_<dataset_type>, feature_2_<dataset_type> = get_feature_model2(data_frame)
# X_<dataset_type>, Y_<dataset_type> = 
# Initiate a regression model and train it.
# Print spearman correlation on the predicted output of the dev and test sets.

## CONFIGURATION 3: Fine-Tune a Contextual Embeddings Model: 18 marks
1 Prepare data samples to be for the DL model to consume. Add the code in the `form_data()`. **4 marks**

3 Create the data loader, one each for train/dev/test data_input sample set obtained from `form_input_example()`. **1.5 marks**

4 Initiate `model3` consisting of **atleast** the following 3 components - `base_LM`, a `pooling_layer` and a `dense_layer`. Use appropriate activation function in dense. **Atleast** one layer of `base_LM` should be set to trainable. **5 marks**

6 Initiate the `loss`. **0.5 marks**

7 Fit the `model3`. Use `NUM_EPOCHS = 2`. **MAX_NUM_EPOCHS allowed will be 3**. **2 marks** 

8 Complete the `get_model_predicts()` to obtain predicted scores for input sentence pairs. **3.5 marks** 

9 Print the correlation scores on the dev and test set predictions. **1.5 mark**

Useful References: https://huggingface.co/blog/how-to-train-sentence-transformers 

In [ ]:
def form_data(data_frame):
  """
  Input a data frame and return the dataloder.
  """

def get_model_predicts(data_type, trained_model):
  """
  Input the dataset list and return a list of cosine similarity scores. Use the fitted final_trainable_model for obtaining encodings.
  """

# dataloader_<dataset_type> = form_data(data_frame)
# base_model = 
# layer_ppoling = 
# layer_dense = 
# model3 = 
# loss =

# Fit the model3.
# Print spearman correlation on the predicted output of the dev and test sets.